# Bioinformatics Pipeline - Gene Expression Analysis
## Interactive Jupyter Notebook Version

Based on: **Liu et al. 2025 (PMC11805404)**  
"Characterization of gene expression profiles in Alzheimer's disease and osteoarthritis"

This notebook implements the exact methodology from the paper.

## Setup and Imports
This section prepares the notebook environment. It imports required Python libraries, sets plotting defaults, and ensures the `results/` directory exists so that all figures and tables produced in later steps are saved automatically.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from pipeline import BioinformaticsPipeline
from pipeline.data_loader import load_paper_geo_data

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

os.makedirs("results", exist_ok=True)

print("✓ All packages imported successfully!")

## Step 1: Initialize Pipeline
Create a reusable `BioinformaticsPipeline` object with a fixed random seed for reproducibility. This object provides the methods used throughout the analysis (PCA, DEG analysis, enrichment, validation, reporting).

In [ ]:
pipeline = BioinformaticsPipeline(random_state=42)
print("Pipeline initialized!")

## Step 1: Data Loading
Load gene expression matrices and phenotype labels for Alzheimer's disease (AD) and osteoarthritis (OA) according to Liu et al. 2025. If a local GEO cache is available, it will be used; otherwise data are downloaded and cached for subsequent runs.

Outputs of this step:
- Training expression matrices and labels for AD and OA
- Optional validation cohorts (if present in cache)
- Basic dataset summary printed to the console

In [ ]:
print("\n" + "="*70)
print("BIOINFORMATICS PIPELINE - GENE EXPRESSION ANALYSIS")
print("Based on: Liu et al. 2025 (PMC11805404)")
print("="*70)
print("\nThis pipeline implements the methodology from the paper:")
print("'Characterization of gene expression profiles in")
print("Alzheimer's disease and osteoarthritis'")
print("\n" + "="*70 + "\n")

print("\n### STEP 1: DATA LOADING ###")
try:
    from geo_loader import get_ad_gse_ids, get_oa_gse_ids
    ad_ids, oa_ids = get_ad_gse_ids(), get_oa_gse_ids()
    print(f"AD GSE IDs: {', '.join(ad_ids)}")
    print(f"OA GSE IDs: {', '.join(oa_ids)}")
    print("(Edit geo_ids.txt or set AD_GSE_IDS / OA_GSE_IDS to change. Cache used if present; else download.)")
except Exception:
    pass

print("Loading GEO data...")
(expr_ad, labels_ad, expr_oa, labels_oa,
 expr_ad_val, labels_ad_val, expr_oa_val, labels_oa_val) = load_paper_geo_data(os.environ.get("GEO_CACHE_DIR"))

print(f"  AD training: {expr_ad.shape[0]} samples, {expr_ad.shape[1]} genes")
print(labels_ad.value_counts().to_string())
print(f"  OA training: {expr_oa.shape[0]} samples, {expr_oa.shape[1]} genes")
print(labels_oa.value_counts().to_string())

    from bioinformatics_project.pipeline.pipeline import BioinformaticsPipeline ## Step 2: Quality Control - PCA
Perform principal component analysis (PCA) on the training datasets to assess sample clustering, detect potential outliers, and visualize separation between case and control groups. The first two principal components are plotted and saved.

Outputs of this step:
- PCA plots for AD and OA saved under `results/`

In [ ]:
print("\n### STEP 2: QUALITY CONTROL - PCA ###")

fig_pca_ad, pca_ad = pipeline.perform_pca(
    expr_ad, labels_ad, 
    title="PCA: Alzheimer's Disease Dataset",
    filename="results/pca_alzheimers.png"
)
plt.show()

In [ ]:
fig_pca_oa, pca_oa = pipeline.perform_pca(
    expr_oa, labels_oa,
    title="PCA: Osteoarthritis Dataset",
    filename="results/pca_osteoarthritis.png"
)
plt.show()

## Step 3: Differential Expression Analysis
Identify differentially expressed genes (DEGs) between case and control samples in each disease cohort. Genes are classified by regulation direction using thresholds on p-value and log fold-change, following the paper’s criteria.

Parameters:
- `pvalue_threshold=0.05`
- `logfc_threshold=0.5`

Outputs of this step:
- `results/degs_alzheimers.csv`
- `results/degs_osteoarthritis.csv`

In [ ]:
print("\n### STEP 3: DIFFERENTIAL EXPRESSION ANALYSIS ###")

degs_ad = pipeline.differential_expression_analysis(
    expr_ad, labels_ad,
    pvalue_threshold=0.05,
    logfc_threshold=0.5
)

degs_oa = pipeline.differential_expression_analysis(
    expr_oa, labels_oa,
    pvalue_threshold=0.05,
    logfc_threshold=0.5
)

degs_ad.to_csv("results/degs_alzheimers.csv", index=False)
degs_oa.to_csv("results/degs_osteoarthritis.csv", index=False)
print("✓ DEG results saved to CSV files")

## Step 4: Volcano Plots
Visualize DEG results using volcano plots, which display statistical significance versus effect size and highlight up- and downregulated genes. These plots help quickly assess the genome-wide expression changes.

Outputs of this step:
- Volcano plots for AD and OA saved under `results/`

In [ ]:
print("\n### STEP 4: VOLCANO PLOTS ###")

fig_volcano_ad = pipeline.plot_volcano(
    degs_ad,
    title="Volcano Plot: Alzheimer's Disease",
    filename="results/volcano_alzheimers.png"
)
plt.show()

In [ ]:
fig_volcano_oa = pipeline.plot_volcano(
    degs_oa,
    title="Volcano Plot: Osteoarthritis",
    filename="results/volcano_osteoarthritis.png"
)
plt.show()

## Step 5: Common DEG Analysis
Intersect the DEG lists from AD and OA to find genes with shared dysregulation patterns. A Venn diagram is generated, and the union/intersection lists are exported to facilitate downstream network and pathway analyses.

Outputs of this step:
- Venn diagram image (`results/venn_diagram.png`)
- `results/common_degs.csv` listing shared genes and regulation

In [ ]:
print("\n### STEP 5: COMMON DEG ANALYSIS ###")

pipeline.degs_disease1 = degs_ad
pipeline.degs_disease2 = degs_oa

fig_venn, co_degs = pipeline.find_common_degs(
    degs_ad, degs_oa,
    disease1_name="AD",
    disease2_name="OA"
)
plt.savefig("results/venn_diagram.png", dpi=300, bbox_inches='tight')
print("✓ Venn diagram saved: results/venn_diagram.png")
plt.show()

co_degs_df = pd.DataFrame({
    'Gene': co_degs['all'],
    'Regulation': ['Upregulated'] * len(co_degs['upregulated']) + 
                 ['Downregulated'] * len(co_degs['downregulated'])
})
co_degs_df.to_csv("results/common_degs.csv", index=False)
print("✓ Common DEGs saved: results/common_degs.csv")

## Step 6: PPI Network Construction
Build a protein–protein interaction (PPI) network from the common DEGs and identify core (hub) genes using topology-based ranking. Only the top subset of genes is used to ensure a readable graph.

Parameters (key):
- Up to 30 input genes; PPI neighbors `k=10`

Outputs of this step:
- PPI network visualization (`results/ppi_network.png`)
- Ranked core genes (`results/core_genes.csv`)

In [ ]:
print("\n### STEP 6: PPI NETWORK CONSTRUCTION ###")

core_genes = []
G = None

if len(co_degs['all']) > 0:
    G, core_genes = pipeline.construct_ppi_network(
        co_degs['all'][:min(len(co_degs['all']), 30)],
        k=10
    )
    
    fig_ppi = pipeline.plot_ppi_network(
        G, core_genes,
        filename="results/ppi_network.png"
    )
    plt.show()
    
    core_genes_df = pd.DataFrame({
        'Gene': core_genes,
        'Rank': range(1, len(core_genes) + 1)
    })
    core_genes_df.to_csv("results/core_genes.csv", index=False)
    print("✓ Core genes saved: results/core_genes.csv")
else:
    print("⚠ Warning: No common DEGs found. Skipping PPI network construction.")

## Step 7: Enrichment Analysis
Perform functional enrichment analysis (e.g., GO terms, KEGG pathways) on the common DEGs to identify biological processes and pathways overrepresented among the shared genes.

Outputs of this step:
- Enrichment barplot (`results/enrichment_analysis.png`)
- Per-category enrichment tables saved as `results/enrichment_*.csv`

In [ ]:
print("\n### STEP 7: ENRICHMENT ANALYSIS ###")

enrichment_results = pipeline.run_enrichment_analysis(co_degs['all'])

fig_enrichment = pipeline.plot_enrichment(
    enrichment_results,
    top_n=10,
    filename="results/enrichment_analysis.png"
)
plt.show()

for category, df in enrichment_results.items():
    df.to_csv(f"results/enrichment_{category}.csv", index=False)
print("✓ Enrichment results saved to CSV files")

## Step 8: Core Gene Validation
Evaluate the predictive value of core genes in independent validation cohorts when available. Receiver operating characteristic (ROC) curves and area under the curve (AUC) metrics quantify discrimination between cases and controls. If no external validation is available, internal cross-cohort validation is performed.

Outputs of this step:
- Expression/ROC figures (saved under `results/` with appropriate prefixes)
- `results/auc_scores.csv` summarizing AUCs per gene

In [ ]:
print("\n### STEP 8: CORE GENE VALIDATION ###")

if len(core_genes) > 0:
    has_ad_val = expr_ad_val is not None and getattr(expr_ad_val, "shape", (0,))[0] > 0
    has_oa_val = expr_oa_val is not None and getattr(expr_oa_val, "shape", (0,))[0] > 0
    if has_ad_val or has_oa_val:
        auc_ad, auc_oa = {}, {}
        if has_ad_val:
            core_in_ad = [g for g in core_genes if g in expr_ad_val.columns]
            if core_in_ad:
                _, _, auc_ad = pipeline.validate_core_genes(
                    expr_ad_val, labels_ad_val, core_in_ad,
                    filename_prefix="results/validation_ad"
                )
                plt.show()
        if has_oa_val:
            core_in_oa = [g for g in core_genes if g in expr_oa_val.columns]
            if core_in_oa:
                _, _, auc_oa = pipeline.validate_core_genes(
                    expr_oa_val, labels_oa_val, core_in_oa,
                    filename_prefix="results/validation_oa"
                )
                plt.show()
        rows = []
        for g in core_genes:
            rows.append({
                "Gene": g,
                "AUC_AD": auc_ad.get(g, np.nan),
                "AUC_OA": auc_oa.get(g, np.nan),
            })
        auc_df = pd.DataFrame(rows)
        auc_df.to_csv("results/auc_scores.csv", index=False)
    else:
        expr_combined = pd.concat([expr_ad, expr_oa], axis=0)
        labels_combined = pd.concat([labels_ad, labels_oa], axis=0)
        fig_expr, fig_roc, auc_scores = pipeline.validate_core_genes(
            expr_combined, labels_combined, core_genes,
            filename_prefix="results/validation"
        )
        plt.show()
        auc_df = pd.DataFrame(list(auc_scores.items()), columns=["Gene", "AUC"])
        auc_df = auc_df.sort_values("AUC", ascending=False)
        auc_df.to_csv("results/auc_scores.csv", index=False)
    print("✓ AUC scores saved: results/auc_scores.csv")
else:
    print("⚠ Warning: No core genes available for validation.")

## Step 9: Generating Final Report
Compile a concise, human-readable text report summarizing inputs, parameters, key findings, and pointers to all generated figures and tables. Use this artifact for documentation and reproducibility.

In [ ]:
print("\n### STEP 9: GENERATING FINAL REPORT ###")

pipeline.generate_report("results/analysis_report.txt")

print("\n" + "="*70)
print("ANALYSIS COMPLETE!")
print("="*70)
print("\nAll results have been saved to the 'results/' directory:")
print("  • PCA plots")
print("  • Volcano plots")
print("  • Venn diagrams")
print("  • PPI network visualization")
print("  • Enrichment analysis plots")
print("  • Gene expression validation")
print("  • ROC curves")
print("  • CSV files with detailed results")
print("  • Comprehensive analysis report")
print("\n" + "="*70)

## Summary Statistics
Produce a compact table capturing the main quantitative outcomes of the analysis for quick inspection and record keeping.

In [ ]:
summary = pd.DataFrame({
    'Metric': [
        'AD Samples',
        'OA Samples',
        'Total Genes',
        'AD DEGs',
        'OA DEGs',
        'Common DEGs',
        'Core Genes'
    ],
    'Value': [
        len(expr_ad),
        len(expr_oa),
        expr_ad.shape[1],
        (degs_ad['Regulation'] != 'Not Significant').sum(),
        (degs_oa['Regulation'] != 'Not Significant').sum(),
        len(co_degs['all']),
        len(core_genes)
    ]
})

print("\n" + "="*50)
print("ANALYSIS SUMMARY")
print("="*50)
print(summary.to_string(index=False))
print("="*50)

## Export Results

In [ ]:
print("\nAll results have been saved during the analysis:")
print("  • results/degs_alzheimers.csv")
print("  • results/degs_osteoarthritis.csv")
print("  • results/common_degs.csv")
print("  • results/core_genes.csv")
print("  • results/enrichment_*.csv")
print("  • results/auc_scores.csv")
print("  • results/analysis_report.txt")
print("  • All visualization plots saved to results/")

## Conclusion

This analysis successfully replicated the methodology from Liu et al. 2025 and identified:

1. **Differentially expressed genes** in both AD and OA
2. **Common molecular signatures** between the two diseases
3. **Core hub genes** with potential biomarker value
4. **Enriched pathways** suggesting shared pathological mechanisms

These findings support the hypothesis of a "bone-brain axis" linking neurodegenerative and musculoskeletal disorders.